# UG1334

***

## Install tools

In [0]:
!sudo apt-get -qq install libprotobuf-dev libhdf5-serial-dev libgflags-dev libgoogle-glog-dev liblmdb-dev libleveldb-dev libsnappy-dev dos2unix

## Get source

In [2]:
!git clone https://github.com/Xilinx/Edge-AI-Platform-Tutorials.git

Cloning into 'Edge-AI-Platform-Tutorials'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 2006 (delta 3), reused 2 (delta 0), pack-reused 1991
Receiving objects: 100% (2006/2006), 537.20 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (697/697), done.
Checking out files: 100% (1520/1520), done.


In [0]:
!cp -R /content/Edge-AI-Platform-Tutorials/docs/Darknet-Caffe-Conversion /content/yolo_convertor

***

## Get YOLOv3 model

In [0]:
!wget --quiet https://pjreddie.com/media/files/yolov3.weights -O yolo_convertor/example_yolov3/0_model_darknet/yolov3.weights

***

## Preparation

- ML_DIR is /content/yolo_convertor

In [5]:
%cd yolo_convertor

/content/yolo_convertor


In [0]:
!tar xf caffe-master.tar.gz 
!tar xf darknet_origin.tar.gz

In [0]:
!cat yolov3*deploy.tar.gz.partaa* > yolov3_deploy.tar.gz
!tar xf yolov3_deploy.tar.gz
!rm yolov3*deploy.tar.gz.*

In [0]:
!tar xf example_yolov3/5_file_for_test/calib_data.tar -C example_yolov3/5_file_for_test

In [0]:
!find . -type f -name "*.txt"   -print0 | xargs -0 dos2unix -q
!find . -type f -name "*.data"  -print0 | xargs -0 dos2unix -q
!find . -type f -name "*.cfg"   -print0 | xargs -0 dos2unix -q
!find . -type f -name "*.names" -print0 | xargs -0 dos2unix -q

In [0]:
!for f in $(find $(pwd) -iname "*.relative"); \
do \
  sed -e "s^PATH_TO^$(pwd)^" ${f} > ${f%.relative}; \
done

***

## Build darknet & Caffe

In [0]:
%cd darknet_origin
!make clean
!make -j$(nproc)
%cd ..

***

- BUild Caffe

In [0]:
!cp /content/Makefile.config /content/yolo_convertor/caffe-master/Makefile.config

In [14]:
%cd caffe-master
!make clean
!make -j$(nproc) pycaffe
!make distribute
%cd ..

/content/yolo_convertor/caffe-master
touch python/caffe/proto/__init__.py
PROTOC src/caffe/proto/caffe.proto
PROTOC (python) src/caffe/proto/caffe.proto
CXX .build_release/src/caffe/proto/caffe.pb.cc
CXX src/caffe/solver.cpp
CXX src/caffe/parallel.cpp
CXX src/caffe/solvers/adadelta_solver.cpp
CXX src/caffe/solvers/adam_solver.cpp
CXX src/caffe/solvers/adagrad_solver.cpp
CXX src/caffe/solvers/sgd_solver.cpp
CXX src/caffe/solvers/nesterov_solver.cpp
CXX src/caffe/solvers/rmsprop_solver.cpp
CXX src/caffe/layer_factory.cpp
CXX src/caffe/transformers/yolo_transformer.cpp
CXX src/caffe/blob.cpp
CXX src/caffe/data_transformer.cpp
CXX src/caffe/layers/lrn_layer.cpp
CXX src/caffe/layers/eltwise_layer.cpp
CXX src/caffe/layers/dropout_layer.cpp
CXX src/caffe/layers/concat_layer.cpp
CXX src/caffe/layers/cudnn_sigmoid_layer.cpp
CXX src/caffe/layers/threshold_layer.cpp
CXX src/caffe/layers/reshape_layer.cpp
CXX src/caffe/layers/sigmoid_layer.cpp
CXX src/caffe/layers/infogain_loss_layer.cpp
CXX src/c

In [15]:
# !export CAFFE_ROOT=$ML_DIR/caffe-master
# !export LD_LIBRARY_PATH=$CAFFE_ROOT/distribute/lib:$LD_LIBRARY_PATH
# !export PYTHONPATH=$CAFFE_ROOT/distribute/python:/usr/local/lib/python2.7/dist-packages/numpy/core/include/:$PYTHONPATH

# check the environment
!CAFFE_ROOT=/content/yolo_convertor/caffe-master \
LD_LIBRARY_PATH=$CAFFE_ROOT/distribute/lib:$LD_LIBRARY_PATH \
PYTHONPATH=$CAFFE_ROOT/distribute/python:/usr/local/lib/python2.7/dist-packages/numpy/core/include/:$PYTHONPATH \
python -c "import caffe; print caffe.__file__"

/content/yolo_convertor/caffe-master/distribute/python/caffe/__init__.py


***

## Run

In [16]:
%cd example_yolov3/
!rm results/*
!rm 5_file_for_test/yolov3_*_result.txt

/content/yolo_convertor/example_yolov3
rm: cannot remove '5_file_for_test/yolov3_*_result.txt': No such file or directory


In [17]:
# step 0: Darknet to Caffe conversion
!bash -v ./0_convert.sh 

#!/bin/bash

python ../yolo_convert.py 0_model_darknet/yolov3.cfg  0_model_darknet/yolov3.weights 1_model_caffe/v3.prototxt 1_model_caffe/v3.caffemodel
block:OrderedDict([('type', 'route'), ('layers', '-4')])
block:OrderedDict([('type', 'route'), ('layers', '-1, 61')])
concat_layer: OrderedDict([('bottom', ['layer85-upsample', 'layer61-shortcut']), ('top', 'layer86-concat'), ('name', 'layer86-concat'), ('type', 'Concat')])
block:OrderedDict([('type', 'route'), ('layers', '-4')])
block:OrderedDict([('type', 'route'), ('layers', '-1, 36')])
concat_layer: OrderedDict([('bottom', ['layer97-upsample', 'layer36-shortcut']), ('top', 'layer98-concat'), ('name', 'layer98-concat'), ('type', 'Concat')])
I0919 11:01:11.939419  3301 upgrade_proto.cpp:69] Attempting to upgrade input file specified using deprecated input fields: 1_model_caffe/v3.prototxt
I0919 11:01:11.939518  3301 upgrade_proto.cpp:72] Successfully upgraded file specified using deprecated input fields.
W0919 11:01:11.939535  3301 up

In [22]:
# step 1: test Darknet and Caffe YOLOv3 models
!bash -v 0_test_darknet.sh
!bash -v 1_test_caffe.sh

#!/bin/bash

#mkdir -p $(pwd)/results
#rm $(pwd)/results/*
#rm $(pwd)/5_file_for_test/yolov3_darknet_result.txt

../darknet_origin/darknet detector valid  5_file_for_test/coco.data 0_model_darknet/yolov3.cfg 0_model_darknet/yolov3.weights -out yolov3_results_
results: Using default 'results'
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 

***

## Use DNNDK

- Download _yolo_convertor/example_yolov3/1_model_caffe/v3.caffemodel / prototxt_

In [0]:
# step 2: quantize YOLOv3 Caffe model
!cp 1_model_caffe/v3.caffemodel  ./2_model_for_quantize/
!bash -v 2_quantize.sh 

#!/bin/bash
# Assume DECENT is already installed to /usr/local/bin
decent quantize -model 2_model_for_quantize/v3.prototxt \
                    -weights 2_model_for_quantize/v3.caffemodel \
                    -gpu 0 \
                    -sigmoided_layers layer81-conv,layer93-conv,layer105-conv \
                    -output_dir 3_model_after_quantize \
                    -method 1
2_quantize.sh: line 3: decent: command not found


In [0]:
# step 3: compile ELF file
!cp 3_model_after_quantize/ref_deploy.prototxt 3_model_after_quantize/deploy.prototxt
!bash -v 3_compile.sh

In [0]:
# step 4: prepare the package for the ZCU102 board
cd ..
cp example_yolov3/4_model_elf/dpu_yolo.elf yolov3_deploy/model/
tar -cvf yolov3_deploy.tar ./yolov3_deploy
gzip -v  yolov3_deploy.tar